# Automated Street View Index Calculation Pipeline using Mapillary Street Images

Before running this Jupyter Notebook, it is recommended to configure it to use a GPU. Follow these steps:

1. Go to the "Runtime" menu at the top.
2. Select "Change runtime type" from the dropdown menu.
3. In the "Runtime type" section, choose "Python 3".
4. In the "Hardware accelerator" section, select "GPU".
5. In the "GPU type" section, choose "T4" if available.
6. In the "Runtime shape" section, select "High RAM".

Save the notebook settings after making these selections.

In [ ]:
# Step 1: Install required libraries
# The following libraries are not installed by default in Google Colab, so run these commands:
%pip install transformers==4.30.1
%pip install geopandas==0.13.2
%pip install torch==1.13.1
%pip install vt2geojson==0.2.1
%pip install mercantile==1.2.1
%pip install osmnx==1.4.0
%pip install pygam==0.8.0
%pip install rasterio==1.3.4
%pip install rioxarray==0.13.3
%pip install xarray-spatial==0.3.5
%pip install rasterio==1.3.4
%pip install shapely==2.0.1
%pip install pystac-client==0.6.0
%pip install planetary-computer==0.4.9
%pip install odc-stac==0.3.6

In [ ]:
# Mount Google Drive to Google Colab
# This step allows you to store the scripts from GitHub and the resulting files in your Google Drive.

# Import the necessary library to mount Google Drive
from google.colab import drive

# Prompt for permission to connect Google Drive
# When running this code, Google Colab will ask for permission to access your Google Drive.
# Follow the provided instructions and grant the necessary permissions.
# Mount Google Drive to '/content/drive' directory
drive.mount('/content/drive')

In [ ]:
# Change the current directory to your Google Drive's root folder
%cd /content/drive/MyDrive

In [ ]:
import os

# Check if the GitHub repository is cloned in your Google Drive
# If the 'StreetView-NatureVisibility' folder doesn't exist, this block of code will clone the repository from GitHub
if not os.path.isdir('StreetView-NatureVisibility'):
    # Clone the GitHub repository to your Google Drive
    !git clone https://github.com/Spatial-Data-Science-and-GEO-AI-Lab/StreetView-NatureVisibility.git

In [ ]:
# Change the current directory to the cloned project folder ('StreetView-NatureVisibility')
%cd StreetView-NatureVisibility

In [ ]:
# Set the desired values for the analysis parameters
place = "Amsterdam, Netherlands"    # Specify the location for the analysis
distance = 50                       # Set the distance parameter for the analysis
cut_by_road_centres = 0             # Determine if panoramic images are going to be cropped using the road centres
access_token = 'MLY|'              # Enter the access token for data retrieval
file_name = 'amsterdam-gvi'         # Set the desired filename for the analysis results
max_workers = 6                     # Specify the maximum number of workers for parallel processing

# Modify the values of 'begin' and 'end' to analyze a specific subset of the dataset
# It's recommended to analyze the dataset in batches considering the available session time for your Google Colab account
# Using the GPU T4, it takes approximately 40 minutes to analyze 2,000 points

begin = 0                        # Write None or set 'begin' to a specific value to start the analysis from a particular point in the dataset
end = 100                        # Write None or set 'end' to a specific value to end the analysis at a particular point in the dataset

In [ ]:
# Run the script to obtain Green View Index (GVI) values
command = f"python main_script.py '{place}' {distance} {cut_by_road_centres} '{access_token}' {file_name} {max_workers} {begin if begin is not None else ''} {end if end is not None else ''}"
!{command}

After running the analysis, you can access the generated results in your Google Drive. Navigate to the 'results' folder, where you will find a subfolder named after the analyzed place. Inside this folder, you will find the following directories:

- **roads**: Contains the GeoPackage (GPKG) file with the road network of the analyzed place.
- **points**: Contains the GeoPackage (GPKG) file with the sampled points.
- **gvi**: Currently, this directory contains the CSV file generated in the previous step.

To obtain the GeoPackage (GPKG) file with the points and their calculated Green View Index (GVI) values, run the following cell.

In [ ]:
# Generate GeoPackage files from the CSV files obtained in the previous step
# The GeoPackage file will be stored in the following location: StreetView-NatureVisibility/results/{place}/gvi/gvi-points.gpkg
command = f"python scripts/get_gvi_gpkg.py '{place}'"
!{command}

Before running the code in the next cell, please note that the Coordinate Reference System (CRS) specified in the code, which is EPSG:32631, is just an example for Amsterdam, Netherlands. It is crucial to change the ESPG code depending on the specific area you are analyzing.

Make sure to update the epsg_code variable with the appropriate projection in meters that is suitable for your study area. For example, you can use the same projection as the one used in the roads.gpkg file.

Additionally, ensure that you have created the required folder structure: StreetView-NatureVisibility/results/{place}/ndvi. Place the corresponding NDVI file, named ndvi.tif, inside this folder. It is recommended to use an NDVI file that has been consistently generated for the study area over the course of a year. The NDVI file must be in the same chosen projection for your area of study.

In [ ]:
# Please modify the code accordingly before executing this cell to ensure accurate results for your specific area of analysis.
# Indicate a projection in meters, suitable for the study area

epsg_code = 32631
file_exists = 0 # Indicates if the file with the calculated ndvi values already exists

In [ ]:
# Changes a line in pygam library to solve a bug
!cp -v pygam/pygam.py /usr/local/lib/python3.10/dist-packages/pygam/pygam.py

In [ ]:
# Compute missing GVI points using NDVI values
# The estimation for the missing points will be stored in the following path: StreetView-NatureVisibility/results/{place}/ndvi/calculated_missing_values_linreg.gpkg and StreetView-NatureVisibility/results/{place}/ndvi/calculated_missing_values_gam.gpkg 
 
command = f"python predict_missing_gvi.py '{place}' {file_exists} {epsg_code} {distance}"
!{command}

Next, it is possible to get the following metrics for the city:

1. Roads types with the most missing images
2. Missing images
3. Panoramic images
4. Image Availability Score and Adjusted Image Availability Score
5. Image Usability Score and Ajdusted Image Usability Score

After running the second code cell, there will be a plot in the results/{place} folder illustrating the roads with the highest amount of missing images in the analysed place.

In [ ]:
# First,we compute the mean Green View Index (GVI) value per street in the road network
# The GeoPackage file will be stored in the following location: StreetView-NatureVisibility/results/{place}/gvi/gvi-streets.gpkg
command = f"python scripts/mean_gvi_street.py '{place}'"
!{command}

In [ ]:
# Now we can get the statistics for the analysed place
command = f"python scripts/results_metrics.py '{place}'"
!{command}